In [ ]:
#%%capture
!pip install bitsandbytes
!pip install transformers==4.34.0
!pip install -U peft
!pip install accelerate
!pip install -U trl
!pip install datasets==2.16.0
!pip install wandb

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch
from datasets import load_dataset
from trl import SFTTrainer

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer,pipeline
from peft import PeftModel
import torch

In [ ]:
pipe = pipeline(
    "text-generation",
    model="DataIntelligenceTeam/mistral_7B_NER",
    #tokenizer = tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

adapter_config.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Result1

In [ ]:

prompt = f'''
Your task is to extract the information corresponding to the provided labels from the below given email.

### Labels:
* pickup_location: Street address of the origin location of goods.
* pickup_cap: Postal code or ZIP code of the pickup location.
* pickup_port: Port of pickup, often used in international shipping.
* pickup_state: Only Country of pickup location.
* delivery_location: Street address of the destination location of goods.
* delivery_cap: Postal code or ZIP code of delivery location.
* delivery_port: Port of delivery, similar to pickup port.
* delivery_state: State or region of delivery location.
* total_quantity: Overall quantity of shipped items (e.g., pieces, boxes). Calculate the total_quantity by summing the quantity of all packages.
* total_weight: Total weight of the shipment (e.g., kg, lbs). Calculate the total_weight by summing the weights of all packages.
* total_volume: Total volume of the shipment (e.g., cubic meters, cubic feet). Calculate the total_volume by summing the volumes of all packages.
* quantity: Individual Quantity of a specific item being shipped.
* package_type: Individual Type of packaging used (e.g., pallets, cartons).
* weight: Individual Weight of a specific package.
* measures: Individual Dimensions or measurements of a package.
* stackable: Indicates whether the shipment is stackable (True or False).
* volume: Individual Volume of a specific package.
* commodity: Type of goods or commodities being shipped.
* company: Name of the email sending company, also the shipping company or carrier.
* incoterms: Choose available options: EXW, FCA, FAS, FOB, CFR, CIF, CPT, CIP, DAP, DPU, DDP.

For attributes with multiple values, such as measures, volume, weight, package_type, and quantity, provide each value separately in a JSON format.

### Input data:
dubai Dear Team, ; Please quote SEA option for below inquiry ; POD :Jebel Ali . ; estimated packing details ;  3300x950x1350 mm ; 3800x1700x1350 mm ; 1900x2000x1200 mm ; 1900x2000x1200 mm ; 2600x1300x1200 mm ; 3900x1100x1350 mm ; 820x780x950 mm ; Total weight kg 5000 approx ; Not stackable ; Marrone Srl ; via Rui, 5 - Loc. Prà dei Risi ; I - 33080 Zoppola (PN) ; Warehouse: 08:00 – 12:00 / 13:00 – 16:30 ; Warehouse Handling + Free days  w.e.f  01-01-2024 ; AED 250 + vat until 10 CBM -  AED 350 + Vat 11 CBM on wards ; Free time applicable - 7 days only for General cargo from DO readiness & No free time for HAZ CARGO, post which standard storage tariff will apply. ; Important Notice ;  : Considering the ongoing circumstances, please be informed after booking confirmation the rates and schedules may undergo changes without advance notice. ; Please also expect vessel delays and changes in transit time. ; CSS will not be liable for any claims that arise due to this. ; Deepak Unnikrishnan ; Sr Sales Coordinator ; Consolidated Shipping Service L.L.C. ; Office: +971 4 883 1303 | Ext: 1163| Toll Free: 800277 | Mobile: +971501513697| ; Dubai | United Arab Emirates ; nvo-import5@cssdubai.com | www.cssgroupsite.com ; Our Offices: Dubai | Abu Dhabi | Sharjah | Ras-Al-Khaimah | Bahrain | Oman | Qatar | Saudi Arabia | Kuwait | Iraq | Africa | Turkiye | India | Sri Lanka | ; Disclaimer: This email and any attachments are confidential and may also be privileged. If you are not the intended recipient, please delete all copies and notify the sender immediately. ; Please read our ; disclaimer.

### Output:
'''

result = pipe(
    f"<s>[INST] {prompt} [/INST]",
    do_sample=True,
    max_new_tokens=32000,
    temperature=0.1,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1,
)
print(result[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST] 
Your task is to extract the information corresponding to the provided labels from the below given email.

### Labels:
* pickup_location: Street address of the origin location of goods.
* pickup_cap: Postal code or ZIP code of the pickup location.
* pickup_port: Port of pickup, often used in international shipping.
* pickup_state: Only Country of pickup location.
* delivery_location: Street address of the destination location of goods.
* delivery_cap: Postal code or ZIP code of delivery location.
* delivery_port: Port of delivery, similar to pickup port.
* delivery_state: State or region of delivery location.
* total_quantity: Overall quantity of shipped items (e.g., pieces, boxes). Calculate the total_quantity by summing the quantity of all packages.
* total_weight: Total weight of the shipment (e.g., kg, lbs). Calculate the total_weight by summing the weights of all packages.
* total_volume: Total volume of the shipment (e.g., cubic meters, cubic feet). Calculate the total_

### Result2

In [ ]:

prompt = f'''
Your task is to extract the information corresponding to the provided labels from the below given email.

### Labels:
* pickup_location: Street address of the origin location of goods.
* pickup_cap: Postal code or ZIP code of the pickup location.
* pickup_port: Port of pickup, often used in international shipping.
* pickup_state: Only Country of pickup location.
* delivery_location: Street address of the destination location of goods.
* delivery_cap: Postal code or ZIP code of delivery location.
* delivery_port: Port of delivery, similar to pickup port.
* delivery_state: State or region of delivery location.
* total_quantity: Overall quantity of shipped items (e.g., pieces, boxes). Calculate the total_quantity by summing the quantity of all packages.
* total_weight: Total weight of the shipment (e.g., kg, lbs). Calculate the total_weight by summing the weights of all packages.
* total_volume: Total volume of the shipment (e.g., cubic meters, cubic feet). Calculate the total_volume by summing the volumes of all packages.
* quantity: Individual Quantity of a specific item being shipped.
* package_type: Individual Type of packaging used (e.g., pallets, cartons).
* weight: Individual Weight of a specific package.
* measures: Individual Dimensions or measurements of a package.
* stackable: Indicates whether the shipment is stackable (True or False).
* volume: Individual Volume of a specific package.
* commodity: Type of goods or commodities being shipped.
* company: Name of the email sending company, also the shipping company or carrier.
* incoterms: Choose available options: EXW, FCA, FAS, FOB, CFR, CIF, CPT, CIP, DAP, DPU, DDP.

For attributes with multiple values, such as measures, volume, weight, package_type, and quantity, provide each value separately in a JSON format.

### Input data:
I: CIC/2324/806 // Rates Ex Italy To Chennai Port Dear Ceriana , ; Please quote your best rate Ex-Works Cesena Italy, By Sea to Chennai Sea Port for below given details : ; Cargo is non stackable and non tiltable, send quote accordingly. ; ITEM  -  EQUIPMENT FOR PHYSICAL EXERCISE ; Total   -   10 Packages ; Net Weight  -  1838.62 Kgs ; Total Weight  -  2291.85 Kgs ; Total Volume (m3)  -  17.36 ; Refer attached Packing List for Dimensions, etc.   ; Pick-Up Address : ; Technogym S.P.A ; Via Calcinaro, 2861 ; 47521 Cesena ( FC) ; Italy ; Thanks & Regards, ; RAKSHANA.R ; A Group of RADAR VENTURES PVT LTD. ; 6th Floor, Menon Eternity, St. Marys Road, ; Alwarpet, Chennai-600018 ; PH NO:7305888727 ; MAIL ID : ; rakshana.r@magikcargo.com

### Output:
'''

result = pipe(
    f"<s>[INST] {prompt} [/INST]",
    do_sample=True,
    max_new_tokens=32000,
    temperature=0.1,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1,
)
print(result[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST] 
Your task is to extract the information corresponding to the provided labels from the below given email.

### Labels:
* pickup_location: Street address of the origin location of goods.
* pickup_cap: Postal code or ZIP code of the pickup location.
* pickup_port: Port of pickup, often used in international shipping.
* pickup_state: Only Country of pickup location.
* delivery_location: Street address of the destination location of goods.
* delivery_cap: Postal code or ZIP code of delivery location.
* delivery_port: Port of delivery, similar to pickup port.
* delivery_state: State or region of delivery location.
* total_quantity: Overall quantity of shipped items (e.g., pieces, boxes). Calculate the total_quantity by summing the quantity of all packages.
* total_weight: Total weight of the shipment (e.g., kg, lbs). Calculate the total_weight by summing the weights of all packages.
* total_volume: Total volume of the shipment (e.g., cubic meters, cubic feet). Calculate the total_

### Result4

In [ ]:

prompt = f'''
Your task is to extract the information corresponding to the provided labels from the below given email.

### Labels:
* pickup_location: Street address of the origin location of goods.
* pickup_cap: Postal code or ZIP code of the pickup location.
* pickup_port: Port of pickup, often used in international shipping.
* pickup_state: Only Country of pickup location.
* delivery_location: Street address of the destination location of goods.
* delivery_cap: Postal code or ZIP code of delivery location.
* delivery_port: Port of delivery, similar to pickup port.
* delivery_state: State or region of delivery location.
* total_quantity: Overall quantity of shipped items (e.g., pieces, boxes). Calculate the total_quantity by summing the quantity of all packages.
* total_weight: Total weight of the shipment (e.g., kg, lbs). Calculate the total_weight by summing the weights of all packages.
* total_volume: Total volume of the shipment (e.g., cubic meters, cubic feet). Calculate the total_volume by summing the volumes of all packages.
* quantity: Individual Quantity of a specific item being shipped.
* package_type: Individual Type of packaging used (e.g., pallets, cartons).
* weight: Individual Weight of a specific package.
* measures: Individual Dimensions or measurements of a package.
* stackable: Indicates whether the shipment is stackable (True or False).
* volume: Individual Volume of a specific package.
* commodity: Type of goods or commodities being shipped.
* company: Name of the email sending company, also the shipping company or carrier.
* incoterms: Choose available options: EXW, FCA, FAS, FOB, CFR, CIF, CPT, CIP, DAP, DPU, DDP.

For attributes with multiple values, such as measures, volume, weight, package_type, and quantity, provide each value separately in a JSON format.

### Input data:
I: CIC/2324/806 // Rates Ex Italy To Chennai Port Dear Ceriana , ; Please quote your best rate Ex-Works Cesena Italy, By Sea to Chennai Sea Port for below given details : ; Cargo is non stackable and non tiltable, send quote accordingly. ; ITEM  -  EQUIPMENT FOR PHYSICAL EXERCISE ; Total   -   10 Packages ; Net Weight  -  1838.62 Kgs ; Total Weight  -  2291.85 Kgs ; Total Volume (m3)  -  17.36 ; Refer attached Packing List for Dimensions, etc.   ; Pick-Up Address : ; Technogym S.P.A ; Via Calcinaro, 2861 ; 47521 Cesena ( FC) ; Italy ; Thanks & Regards, ; RAKSHANA.R ; A Group of RADAR VENTURES PVT LTD. ; 6th Floor, Menon Eternity, St. Marys Road, ; Alwarpet, Chennai-600018 ; PH NO:7305888727 ; MAIL ID : ; rakshana.r@magikcargo.com

### Output:
'''

result = pipe(
    f"<s>[INST] {prompt} [/INST]",
    do_sample=True,
    max_new_tokens=32000,
    temperature=0.1,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1,
)
print(result[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST] 
Your task is to extract the information corresponding to the provided labels from the below given email.

### Labels:
* pickup_location: Street address of the origin location of goods.
* pickup_cap: Postal code or ZIP code of the pickup location.
* pickup_port: Port of pickup, often used in international shipping.
* pickup_state: Only Country of pickup location.
* delivery_location: Street address of the destination location of goods.
* delivery_cap: Postal code or ZIP code of delivery location.
* delivery_port: Port of delivery, similar to pickup port.
* delivery_state: State or region of delivery location.
* total_quantity: Overall quantity of shipped items (e.g., pieces, boxes). Calculate the total_quantity by summing the quantity of all packages.
* total_weight: Total weight of the shipment (e.g., kg, lbs). Calculate the total_weight by summing the weights of all packages.
* total_volume: Total volume of the shipment (e.g., cubic meters, cubic feet). Calculate the total_

### Result5

In [ ]:

prompt = f'''
Your task is to extract the information corresponding to the provided labels from the below given email.

### Labels:
* pickup_location: Street address of the origin location of goods.
* pickup_cap: Postal code or ZIP code of the pickup location.
* pickup_port: Port of pickup, often used in international shipping.
* pickup_state: Only Country of pickup location.
* delivery_location: Street address of the destination location of goods.
* delivery_cap: Postal code or ZIP code of delivery location.
* delivery_port: Port of delivery, similar to pickup port.
* delivery_state: State or region of delivery location.
* total_quantity: Overall quantity of shipped items (e.g., pieces, boxes). Calculate the total_quantity by summing the quantity of all packages.
* total_weight: Total weight of the shipment (e.g., kg, lbs). Calculate the total_weight by summing the weights of all packages.
* total_volume: Total volume of the shipment (e.g., cubic meters, cubic feet). Calculate the total_volume by summing the volumes of all packages.
* quantity: Individual Quantity of a specific item being shipped.
* package_type: Individual Type of packaging used (e.g., pallets, cartons).
* weight: Individual Weight of a specific package.
* measures: Individual Dimensions or measurements of a package.
* stackable: Indicates whether the shipment is stackable (True or False).
* volume: Individual Volume of a specific package.
* commodity: Type of goods or commodities being shipped.
* company: Name of the email sending company, also the shipping company or carrier.
* incoterms: Choose available options: EXW, FCA, FAS, FOB, CFR, CIF, CPT, CIP, DAP, DPU, DDP.

For attributes with multiple values, such as measures, volume, weight, package_type, and quantity, provide each value separately in a JSON format.

### Input data:
I: KEELUNG Ciao Paolo, ; Per favore mi quoti per: ; Winery: Cantina Tollo ; pick up::66010 ; POD: Keelung ; Cargo size : 120x 100x190 cm/ 2 Pallets 1581/Kg ; 210 cases ,1581kg, 2Pallets about 4.56 CBM ; Grazie ; Cordiali saluti/ Best regards ; ________________________________________________________________________________ ; Franco Raiola  – Area manager ; Cargo Compass spa ; Via Lavoria 56/L/M/N | Cenaia | Crespina Lorenzana | Pisa | 56040  | ITALY ; fraiola@cargocompass.it | O +39.050.643133  | F +39.050.644687 | C+39.335.1277205  | www.cargocompassworld.com

### Output:
'''

result = pipe(
    f"<s>[INST] {prompt} [/INST]",
    do_sample=True,
    max_new_tokens=32000,
    temperature=0.1,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1,
)
print(result[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST] 
Your task is to extract the information corresponding to the provided labels from the below given email.

### Labels:
* pickup_location: Street address of the origin location of goods.
* pickup_cap: Postal code or ZIP code of the pickup location.
* pickup_port: Port of pickup, often used in international shipping.
* pickup_state: Only Country of pickup location.
* delivery_location: Street address of the destination location of goods.
* delivery_cap: Postal code or ZIP code of delivery location.
* delivery_port: Port of delivery, similar to pickup port.
* delivery_state: State or region of delivery location.
* total_quantity: Overall quantity of shipped items (e.g., pieces, boxes). Calculate the total_quantity by summing the quantity of all packages.
* total_weight: Total weight of the shipment (e.g., kg, lbs). Calculate the total_weight by summing the weights of all packages.
* total_volume: Total volume of the shipment (e.g., cubic meters, cubic feet). Calculate the total_

### Result6

In [ ]:

prompt = f'''
Your task is to extract the information corresponding to the provided labels from the below given email.

### Labels:
* pickup_location: Street address of the origin location of goods.
* pickup_cap: Postal code or ZIP code of the pickup location.
* pickup_port: Port of pickup, often used in international shipping.
* pickup_state: Only Country of pickup location.
* delivery_location: Street address of the destination location of goods.
* delivery_cap: Postal code or ZIP code of delivery location.
* delivery_port: Port of delivery, similar to pickup port.
* delivery_state: State or region of delivery location.
* total_quantity: Overall quantity of shipped items (e.g., pieces, boxes). Calculate the total_quantity by summing the quantity of all packages.
* total_weight: Total weight of the shipment (e.g., kg, lbs). Calculate the total_weight by summing the weights of all packages.
* total_volume: Total volume of the shipment (e.g., cubic meters, cubic feet). Calculate the total_volume by summing the volumes of all packages.
* quantity: Individual Quantity of a specific item being shipped.
* package_type: Individual Type of packaging used (e.g., pallets, cartons).
* weight: Individual Weight of a specific package.
* measures: Individual Dimensions or measurements of a package.
* stackable: Indicates whether the shipment is stackable (True or False).
* volume: Individual Volume of a specific package.
* commodity: Type of goods or commodities being shipped.
* company: Name of the email sending company, also the shipping company or carrier.
* incoterms: Choose available options: EXW, FCA, FAS, FOB, CFR, CIF, CPT, CIP, DAP, DPU, DDP.

For attributes with multiple values, such as measures, volume, weight, package_type, and quantity, provide each value separately in a JSON format.

### Input data:
I: LAINATE > KEE Ciao Paolo / Giorgia, ; Per favore mi quotate da LAINATE (MI)  a KEELUNG ; (MOBILI) ; 5 colli – 168,7 kg ; 120x80x187 ; 265x100x78 ; 120x53x54 ; 78x82x78 ; 63x56x100 ; Grazie ciao   ; Alberto Porta ; MOBILE NUMBER + 39 3337934630 ; T: +39 0331 1090068 ; E: alberto.porta@nordestgroup.it ; W: www.nordestgroup.it ; Branch MXP: Via del Gregge n°100 – 21015  Lonate Pozzolo ( VA ) ; IATA code 38471480003 – PI/VAT 01327440937 – SDI M5UXCR1 ; Headquarters: Viale Ponte Rosso 19 - 33078 San Vito al Tagliamento ( PN ) ; Branch GOA: Piazza della Nunziata n°5 - 16124 Genova ( GE )

### Output:
'''

result = pipe(
    f"<s>[INST] {prompt} [/INST]",
    do_sample=True,
    max_new_tokens=32000,
    temperature=0.1,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1,
)
print(result[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST] 
Your task is to extract the information corresponding to the provided labels from the below given email.

### Labels:
* pickup_location: Street address of the origin location of goods.
* pickup_cap: Postal code or ZIP code of the pickup location.
* pickup_port: Port of pickup, often used in international shipping.
* pickup_state: Only Country of pickup location.
* delivery_location: Street address of the destination location of goods.
* delivery_cap: Postal code or ZIP code of delivery location.
* delivery_port: Port of delivery, similar to pickup port.
* delivery_state: State or region of delivery location.
* total_quantity: Overall quantity of shipped items (e.g., pieces, boxes). Calculate the total_quantity by summing the quantity of all packages.
* total_weight: Total weight of the shipment (e.g., kg, lbs). Calculate the total_weight by summing the weights of all packages.
* total_volume: Total volume of the shipment (e.g., cubic meters, cubic feet). Calculate the total_

### Result7

In [ ]:

prompt = f'''
Your task is to extract the information corresponding to the provided labels from the below given email.

### Labels:
* pickup_location: Street address of the origin location of goods.
* pickup_cap: Postal code or ZIP code of the pickup location.
* pickup_port: Port of pickup, often used in international shipping.
* pickup_state: Only Country of pickup location.
* delivery_location: Street address of the destination location of goods.
* delivery_cap: Postal code or ZIP code of delivery location.
* delivery_port: Port of delivery, similar to pickup port.
* delivery_state: State or region of delivery location.
* total_quantity: Overall quantity of shipped items (e.g., pieces, boxes). Calculate the total_quantity by summing the quantity of all packages.
* total_weight: Total weight of the shipment (e.g., kg, lbs). Calculate the total_weight by summing the weights of all packages.
* total_volume: Total volume of the shipment (e.g., cubic meters, cubic feet). Calculate the total_volume by summing the volumes of all packages.
* quantity: Individual Quantity of a specific item being shipped.
* package_type: Individual Type of packaging used (e.g., pallets, cartons).
* weight: Individual Weight of a specific package.
* measures: Individual Dimensions or measurements of a package.
* stackable: Indicates whether the shipment is stackable (True or False).
* volume: Individual Volume of a specific package.
* commodity: Type of goods or commodities being shipped.
* company: Name of the email sending company, also the shipping company or carrier.
* incoterms: Choose available options: EXW, FCA, FAS, FOB, CFR, CIF, CPT, CIP, DAP, DPU, DDP.

For attributes with multiple values, such as measures, volume, weight, package_type, and quantity, provide each value separately in a JSON format.

### Input data:
I: RATE EXW FRANCE - BUENOS AIRES LCL qt3218 Dear Davide, ; Is it possible coordinate this cargo for Genova? ; PICK UP LOCATION: ; 6 rue Désir Prévost – La grande Brénche ; 91070 Bondoufle FRANCE ; DESTINATION: BUENOS AIRES ; CARGO : ; 1 pallet / 120x80x123 cm – 124 kgs ; 1 pallet / 120x80x108 cm – 108 kgs ; Caga general ; Brgds, ; Estèfany

### Output:
'''

result = pipe(
    f"<s>[INST] {prompt} [/INST]",
    do_sample=True,
    max_new_tokens=32000,
    temperature=0.1,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1,
)
print(result[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST] 
Your task is to extract the information corresponding to the provided labels from the below given email.

### Labels:
* pickup_location: Street address of the origin location of goods.
* pickup_cap: Postal code or ZIP code of the pickup location.
* pickup_port: Port of pickup, often used in international shipping.
* pickup_state: Only Country of pickup location.
* delivery_location: Street address of the destination location of goods.
* delivery_cap: Postal code or ZIP code of delivery location.
* delivery_port: Port of delivery, similar to pickup port.
* delivery_state: State or region of delivery location.
* total_quantity: Overall quantity of shipped items (e.g., pieces, boxes). Calculate the total_quantity by summing the quantity of all packages.
* total_weight: Total weight of the shipment (e.g., kg, lbs). Calculate the total_weight by summing the weights of all packages.
* total_volume: Total volume of the shipment (e.g., cubic meters, cubic feet). Calculate the total_

### Result8

In [ ]:

prompt = f'''
Your task is to extract the information corresponding to the provided labels from the below given email.

### Labels:
* pickup_location: Street address of the origin location of goods.
* pickup_cap: Postal code or ZIP code of the pickup location.
* pickup_port: Port of pickup, often used in international shipping.
* pickup_state: Only Country of pickup location.
* delivery_location: Street address of the destination location of goods.
* delivery_cap: Postal code or ZIP code of delivery location.
* delivery_port: Port of delivery, similar to pickup port.
* delivery_state: State or region of delivery location.
* total_quantity: Overall quantity of shipped items (e.g., pieces, boxes). Calculate the total_quantity by summing the quantity of all packages.
* total_weight: Total weight of the shipment (e.g., kg, lbs). Calculate the total_weight by summing the weights of all packages.
* total_volume: Total volume of the shipment (e.g., cubic meters, cubic feet). Calculate the total_volume by summing the volumes of all packages.
* quantity: Individual Quantity of a specific item being shipped.
* package_type: Individual Type of packaging used (e.g., pallets, cartons).
* weight: Individual Weight of a specific package.
* measures: Individual Dimensions or measurements of a package.
* stackable: Indicates whether the shipment is stackable (True or False).
* volume: Individual Volume of a specific package.
* commodity: Type of goods or commodities being shipped.
* company: Name of the email sending company, also the shipping company or carrier.
* incoterms: Choose available options: EXW, FCA, FAS, FOB, CFR, CIF, CPT, CIP, DAP, DPU, DDP.

For attributes with multiple values, such as measures, volume, weight, package_type, and quantity, provide each value separately in a JSON format.

### Input data:
I: RICHIESTA OFFERTA CFR CHITTOGRAM 6 PLT 8166 KG NO ADR Buon pomeriggio, ; chiedo cortesemente vs offerta per ; CFR CHITTOGRAM ; n. 5 pallet – 130x110x200h ; n. 1 pallet – 130x110x50h ; kg. 8166 ; no ADR ; Ritiro in linea: VERDELLINO (BG) ; Grazie  ; Deborah Corbetta ; Sales Representative ; Savino Del Bene S.p.A. – Bergamo ; Airport Center ; Via Orio 29 d/e ; 24050 Grassobbio (BG) ; + 39 035 586 3011 int. 322 – cell. 331 6573947 ; deborah.corbetta@savinodelbene.com  ;   www.savinodelbene.com

### Output:
'''

result = pipe(
    f"<s>[INST] {prompt} [/INST]",
    do_sample=True,
    max_new_tokens=32000,
    temperature=0.1,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1,
)
print(result[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST] 
Your task is to extract the information corresponding to the provided labels from the below given email.

### Labels:
* pickup_location: Street address of the origin location of goods.
* pickup_cap: Postal code or ZIP code of the pickup location.
* pickup_port: Port of pickup, often used in international shipping.
* pickup_state: Only Country of pickup location.
* delivery_location: Street address of the destination location of goods.
* delivery_cap: Postal code or ZIP code of delivery location.
* delivery_port: Port of delivery, similar to pickup port.
* delivery_state: State or region of delivery location.
* total_quantity: Overall quantity of shipped items (e.g., pieces, boxes). Calculate the total_quantity by summing the quantity of all packages.
* total_weight: Total weight of the shipment (e.g., kg, lbs). Calculate the total_weight by summing the weights of all packages.
* total_volume: Total volume of the shipment (e.g., cubic meters, cubic feet). Calculate the total_

### Result 9

In [ ]:

prompt = f'''
Your task is to extract the information corresponding to the provided labels from the below given email.

### Labels:
* pickup_location: Street address of the origin location of goods.
* pickup_cap: Postal code or ZIP code of the pickup location.
* pickup_port: Port of pickup, often used in international shipping.
* pickup_state: Only Country of pickup location.
* delivery_location: Street address of the destination location of goods.
* delivery_cap: Postal code or ZIP code of delivery location.
* delivery_port: Port of delivery, similar to pickup port.
* delivery_state: State or region of delivery location.
* total_quantity: Overall quantity of shipped items (e.g., pieces, boxes). Calculate the total_quantity by summing the quantity of all packages.
* total_weight: Total weight of the shipment (e.g., kg, lbs). Calculate the total_weight by summing the weights of all packages.
* total_volume: Total volume of the shipment (e.g., cubic meters, cubic feet). Calculate the total_volume by summing the volumes of all packages.
* quantity: Individual Quantity of a specific item being shipped.
* package_type: Individual Type of packaging used (e.g., pallets, cartons).
* weight: Individual Weight of a specific package.
* measures: Individual Dimensions or measurements of a package.
* stackable: Indicates whether the shipment is stackable (True or False).
* volume: Individual Volume of a specific package.
* commodity: Type of goods or commodities being shipped.
* company: Name of the email sending company, also the shipping company or carrier.
* incoterms: Choose available options: EXW, FCA, FAS, FOB, CFR, CIF, CPT, CIP, DAP, DPU, DDP.

For attributes with multiple values, such as measures, volume, weight, package_type, and quantity, provide each value separately in a JSON format.

### Input data:
I: IN-38521 // ITALIA- MEX // LCL VAN BERKEL Gentilissimi, ; mio agente Messicano (Inndingo)  ha negoziato con la Vs. Messicana (Mewtral Mexico) nolo in Collect. ; Mi occorrono costi di ritiro+magazzino+operazione doganale per: ; 1plts cm 83*71*83 - 64 kg - ; Destino Veracruz CFS ; Ritiro ; Van Berkel International Srl ; C/O Interlaziale ; Via Milano 10 ; Ceriano Laghetto (MB) – 20816 ; Italia ; Orario magazzino: 8.30-12.30 / 13.30-17.00 – DA LUNEDI A VENERDI ; Flete marítimo en términos collect con mi contrato con la consolidadora Sinergy Cargo (Coloader Newtral Mexico). ; Compárteme tus costos de recolección, handling, despacho aduanal, costo de EUR-1, y costos pertinentes a este envío, así como próximas salidas, gracias! ; Se occorrono altri dati rimango a disposizione. ; Alessio

### Output:
'''

result = pipe(
    f"<s>[INST] {prompt} [/INST]",
    do_sample=True,
    max_new_tokens=32000,
    temperature=0.1,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1,
)
print(result[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST] 
Your task is to extract the information corresponding to the provided labels from the below given email.

### Labels:
* pickup_location: Street address of the origin location of goods.
* pickup_cap: Postal code or ZIP code of the pickup location.
* pickup_port: Port of pickup, often used in international shipping.
* pickup_state: Only Country of pickup location.
* delivery_location: Street address of the destination location of goods.
* delivery_cap: Postal code or ZIP code of delivery location.
* delivery_port: Port of delivery, similar to pickup port.
* delivery_state: State or region of delivery location.
* total_quantity: Overall quantity of shipped items (e.g., pieces, boxes). Calculate the total_quantity by summing the quantity of all packages.
* total_weight: Total weight of the shipment (e.g., kg, lbs). Calculate the total_weight by summing the weights of all packages.
* total_volume: Total volume of the shipment (e.g., cubic meters, cubic feet). Calculate the total_